In [28]:
# notebook to execute a list of jobfiles without supervision for "stream not found" or bad exit codes
# repeats same file 5 times then moves on

import os
import subprocess
import shlex
import numpy as np
from select import select
from subprocess import Popen

In [89]:

# directory where ampel job should be run (NEEDS .tmp FOLDER)
execute_directory="/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2"

# directory where jobfiles are found
jobfile_save_dir="/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/examples/calibrateKilonovaEval_jobfiles/"

# list of names of jobfiles to run
jobfile_names=("ligo_healpix_S200115j.yaml", "ligo_healpix_S190426c.yaml")

#config file to use
conf_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/ampel_conf.yaml"

#vault file to use
vault_file = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/vault.yaml"



In [90]:
def execute_jobfile(job_call, execute_directory):
    command = job_call
    command_list = shlex.split(command)

    print(command_list)

    stream_not_found_err = "Stream not found"

    os.chdir(execute_directory)

    timeout = 0.1
    retry = True
    max_retries = 5
    retry_ind = 0
    while retry:
        print("STARTING SUBPROCESS, try #", retry_ind)
        try:
            process = Popen(command_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except NameError:
            print("Error")
        retry = False

        
        while process:
            #output, error = process.communicate()
            #print(output, error)

            if process.poll() is not None:
                if process.returncode != 0:
                    print("BAD EXIT")
                    print(process.stderr.read().decode("utf-8"), end="")
                    retry = True
                    break
                print(process.stdout.read().decode("utf-8"), end="")
                process.stdout.close()
                break
            
            rlist = select([process.stdout], [], [], timeout)[0]

            for f in rlist:
                line = f.readline().decode("utf-8")
                print(line, end="")
                if line.find(stream_not_found_err) != -1:
                    print("FOUND", stream_not_found_err)
                    retry = True
                    process.kill()
                #elif line.find("Error") != -1:
                    #print("Found SyntaxError")
                    #retry = True
                    #process.kill()
                #print(f"{f.readline()}")

        

        retry_ind += 1
        if retry_ind >= max_retries:
            print("MAX RETRIES REACHED, STOPPING")
            break

        print("RETRY?", retry)

In [91]:
for jobfile_name in jobfile_names:

	jobfile = os.path.join(jobfile_save_dir, jobfile_name)
	job_call = "" #f"cd {execute_directory};"
	job_call += f"ampel job --schema {jobfile} --config {conf_file} --secrets {vault_file}"

	execute_jobfile(job_call, execute_directory)

['ampel', 'job', '--schema', '/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/examples/calibrateKilonovaEval_jobfiles/ligo_healpix_S200115j.yaml', '--config', '/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/ampel_conf.yaml', '--secrets', '/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/Ampel-HU-astro/vault.yaml']
STARTING SUBPROCESS, try # 0
 ----------------

2023-06-27 15:00:40 CORE T3Processor:76 SHOUT
 Running ligo#0

2023-06-27 15:00:40 CORE T3Processor:119 INFO [u=T3PlainUnitExecutor]
 Processing run block 0

2023-06-27 15:00:40 CORE T3PlainUnitExecutor:76 INFO [u=HealpixTokenGenerator]
 Running T3 unit
{'jd': {'$gt': 2458863.682751548, '$lt': 2458866.682751548}, 'regions': [{'nside': 16, 'pixels': [1801, 1793, 2, 1795, 1934, 121, 103, 1803, 1932, 2960, 1825, 1959]}, {'nside': 32, 'pixels': [11844, 2, 7685, 7687, 12, 7693, 14, 15, 504, 11807, 11921, 7698, 483, 411, 7704, 26, 9198, 11805, 7830, 7740, 33, 7201, 7203